<a href="https://colab.research.google.com/github/luanfelixcoding/climate-change-impact-economy/blob/Mafra/climate_change_impact_economy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link para base de dados: https://www.kaggle.com/datasets/tarunrm09/climate-change-indicators

link para base de dados: https://www.kaggle.com/datasets/sazidthe1/global-inflation-data

In [40]:
# conectando com o drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# importando bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [42]:
climate_change_file_path: str = '/content/drive/My Drive/datasets/climate_change_dataset/climate_change_indicators.csv'
global_inflation_file_path : str = '/content/drive/My Drive/datasets/global_inflation_dataset/global_inflation_data.csv'

# datasets
df_climate = pd.read_csv(climate_change_file_path)
df_economy = pd.read_csv(global_inflation_file_path)

In [43]:
df_climate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225 entries, 0 to 224
Data columns (total 72 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ObjectId             225 non-null    int64  
 1   Country              225 non-null    object 
 2   ISO2                 223 non-null    object 
 3   ISO3                 225 non-null    object 
 4   Indicator            225 non-null    object 
 5   Unit                 225 non-null    object 
 6   Source               225 non-null    object 
 7   CTS_Code             225 non-null    object 
 8   CTS_Name             225 non-null    object 
 9   CTS_Full_Descriptor  225 non-null    object 
 10  F1961                188 non-null    float64
 11  F1962                189 non-null    float64
 12  F1963                188 non-null    float64
 13  F1964                188 non-null    float64
 14  F1965                188 non-null    float64
 15  F1966                192 non-null    flo

In [44]:
df_economy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country_name    196 non-null    object 
 1   indicator_name  196 non-null    object 
 2   1980            140 non-null    float64
 3   1981            144 non-null    float64
 4   1982            145 non-null    float64
 5   1983            145 non-null    float64
 6   1984            145 non-null    float64
 7   1985            145 non-null    float64
 8   1986            145 non-null    float64
 9   1987            147 non-null    float64
 10  1988            147 non-null    float64
 11  1989            147 non-null    float64
 12  1990            150 non-null    float64
 13  1991            155 non-null    float64
 14  1992            158 non-null    float64
 15  1993            169 non-null    float64
 16  1994            171 non-null    float64
 17  1995            172 non-null    flo

In [45]:
# verificando valores nulos em cada coluna
def checking_null_values(df: pd.DataFrame) -> None:
  for col in df.columns:
    if df[col].isnull().sum() > 0:
      print(f'{df[col].name} = {df[col].isnull().sum()}')

In [46]:
# Useless code block

selected_rows = df_climate[df_climate['Country'] == 'Brazil']
selected_rows

,ObjectId,Country,ISO2,ISO3,Indicator,Unit,Source,CTS_Code,CTS_Name,CTS_Full_Descriptor,...,F2013,F2014,F2015,F2016,F2017,F2018,F2019,F2020,F2021,F2022
26,27,Brazil,BR,BRA,Temperature change with respect to a baseline ...,Degree Celsius,Food and Agriculture Organization of the Unite...,ECCS,Surface Temperature Change,"Environment, Climate Change, Climate Indicator...",...,0.922,1.153,1.516,1.457,1.363,1.148,1.517,1.477,1.154,0.926


In [47]:
# Pegando apenas as colunas que representam anos
years = [col for col in df_climate.columns if col.startswith('F')]

# Derrete o dataset em variaveis e valores
# 'Country' e 'ISO3' mantemos, nomeamos uma coluna como 'Year' que recebe a variavel 'years'
df_melted = df_climate.melt(id_vars=['Country', 'ISO3'], value_vars=years, var_name='Year', value_name='Temperature_Change')

# Na coluna 'Year' extraindo todos os numeros e convertendo para int, '(\d+)' permite extrair numeros compostos
df_melted['Year'] = df_melted['Year'].str.extract(r'(\d+)').astype(int)
# checking_null_values(df_melted) # output: Temperature_Change = 1490


# Removendo valores nulos; inplace=True faz as alterações no proprio df, inplace=False cria outro df com os valores removidos
df_melted.dropna(subset=['Temperature_Change'], inplace=True)
checking_null_values(df_melted) # output: Temperature_Change = 0

## Plotando Gráfico referente as mudanças climáticas durante os anos

In [48]:
# Agrupando a coluna 'Year' e pegando a media de seus valores em conjunto
global_temp_trend = df_melted.groupby('Year', as_index=False)['Temperature_Change'].mean().sort_values(by='Year')

# Criando a visualização
fig_global = go.Figure()

fig_global.add_trace(go.Scatter(
    x=global_temp_trend['Year'],
    y=global_temp_trend['Temperature_Change'],
    mode='lines+markers',
    line=dict(color='red', width=2),  # Linha vermelha
    marker=dict(color='black', size=5),  # Pontos pretos
    name='Tendência Global'
))


fig_global.update_layout(
    title='📈 Tendência Global da Mudança de Temperatura (1961-2022)',
    xaxis_title='Ano',
    yaxis_title='Mudança Média de Temperatura (°C)',
    template='plotly_white',
    showlegend=True
)

# Exibir o gráfico
fig_global.show()

In [49]:
# Transformando o dataset para formato longo
df_melted = df_climate.melt(id_vars=['Country', 'ISO3'], value_vars=years, var_name='Year', value_name='Temperature_Change')

# Extraindo os anos corretamente e convertendo para inteiro
df_melted['Year'] = df_melted['Year'].str.extract(r'(\d+)').astype(int)

# Removendo valores nulos na temperatura
df_melted.dropna(subset=['Temperature_Change'], inplace=True)

# Agrupando por Ano e calculando a média global da mudança de temperatura
global_temp_trend = df_melted.groupby('Year', as_index=False)['Temperature_Change'].mean().sort_values(by='Year')

# Criando a visualização
fig_global = go.Figure()


fig_global.add_trace(go.Scatter(
    x=global_temp_trend['Year'],
    y=global_temp_trend['Temperature_Change'],
    mode='lines+markers',
    line=dict(color='red', width=2),  # Linha vermelha
    marker=dict(color='black', size=4),  # Pontos pretos
    name='Tendência Global'
))


fig_global.update_layout(
    title='📈 Tendência Global da Mudança de Temperatura (1961-2022)',
    xaxis_title='Ano',
    yaxis_title='Mudança Média de Temperatura (°C)',
    template='plotly_white',
    showlegend=True
)

# Exibir o gráfico
fig_global.show()

In [50]:
# Calculando a variação de temperatura por país (máximo - mínimo)
df_variation = df_melted.groupby('Country')['Temperature_Change'].agg(lambda x: x.max() - x.min())

# Selecionando os 10 países mais afetados (com maior variação)
top_countries = df_variation.nlargest(10).index
df_top_countries = df_melted[df_melted['Country'].isin(top_countries)].sort_values(by=['Year'])

# Criando o gráfico interativo
fig_countries = px.line(df_top_countries, x='Year', y='Temperature_Change', color='Country',
                        title='🌍 Comparação dos Países com Maior Variação ao longo dos Anos',
                        labels={'Temperature_Change': 'Mudança Média de Temperatura (°C)', 'Year': 'Ano'},
                        markers=True)

fig_countries.update_traces(line=dict(width=2))
fig_countries.show()


In [51]:
# BLOCO DELETAR INÚTIL - pegando os 10 países com maior temperatura média
# o ideal é que nem o bloco de cima - pegando os países com maior variação ao longo dos anos

top_countries = df_melted.groupby('Country')['Temperature_Change'].mean().nlargest(10).index
df_top_countries = df_melted[df_melted['Country'].isin(top_countries)]
# df_top_countries = df_top_countries.sort_values(by=['Temperature_Change'], ascending=True)

# Comparando os países mais afetados
fig_countries = px.line(df_top_countries, x='Year', y='Temperature_Change', color='Country',
                        title='🌍 Comparação da Mudança de Temperatura para os 10 Países Mais Afetados',
                        labels={'Temperature_Change': 'Mudança Média de Temperatura (°C)', 'Year': 'Ano'},
                        markers=True)

fig_countries.update_traces(line=dict(width=2))
fig_countries.show()

## Tratando dataset Inflação

In [52]:
# Renomear as colunas para facilitar o uso
df_economy = df_economy.rename(columns={'country_name': 'Country', 'indicator_name': 'Indicator'})

In [53]:
# Verificando valores nulos
def checking_null_values(df: pd.DataFrame) -> None:
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            print(f'{df[col].name} = {df[col].isnull().sum()}')

checking_null_values(df_economy)

1980 = 56
1981 = 52
1982 = 51
1983 = 51
1984 = 51
1985 = 51
1986 = 51
1987 = 49
1988 = 49
1989 = 49
1990 = 46
1991 = 41
1992 = 38
1993 = 27
1994 = 25
1995 = 24
1996 = 20
1997 = 17
1998 = 15
1999 = 14
2000 = 13
2001 = 9
2002 = 7
2003 = 6
2004 = 5
2005 = 3
2006 = 3
2007 = 3
2008 = 3
2009 = 3
2010 = 3
2011 = 4
2012 = 3
2013 = 2
2014 = 2
2015 = 2
2016 = 2
2017 = 1
2018 = 1
2019 = 1
2020 = 2
2021 = 2
2022 = 2
2023 = 4
2024 = 5


In [54]:
# Interpolação para preencher valores nulos
for col in df_economy.columns[2:]:  # Começa da terceira coluna (anos)
    df_economy[col].interpolate(method='linear', inplace=True)

<ipython-input-54-4a78a65b49e4>:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [55]:
# Preenchendo o valor nulo em 2024 com a média da coluna
df_economy['2024'].fillna(df_economy['2024'].mean(), inplace=True)

<ipython-input-55-c6a394b52e6e>:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [56]:
# Verificando se ainda há valores nulos
def checking_null_values(df: pd.DataFrame) -> None:
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            print(f'{df[col].name} = {df[col].isnull().sum()}')

checking_null_values(df_economy)

In [57]:
# Transformando para formato longo
years = [col for col in df_economy.columns if col.isdigit()]
df_economy_melted = df_economy.melt(id_vars=['Country', 'Indicator'], value_vars=years, var_name='Year', value_name='Inflation')


In [58]:
# Convertendo anos para inteiro
df_economy_melted['Year'] = df_economy_melted['Year'].astype(int)

## Plotando Gráfico referente a inflação

In [59]:
# Visualização da tendência global da inflação
global_inflation_trend = df_economy_melted.groupby('Year', as_index=False)['Inflation'].mean().sort_values(by='Year')

fig_inflation_global = go.Figure()
fig_inflation_global.add_trace(go.Scatter(
    x=global_inflation_trend['Year'],
    y=global_inflation_trend['Inflation'],
    mode='lines+markers',
    name='Tendência Global da Inflação'
))

fig_inflation_global.update_layout(
    title='📈 Tendência Global da Inflação (1980-2024)',
    xaxis_title='Ano',
    yaxis_title='Inflação Média',
    template='plotly_white',
    showlegend=True
)

fig_inflation_global.show()

In [60]:
# Análise da variação da inflação por país
df_inflation_variation = df_economy_melted.groupby('Country')['Inflation'].agg(lambda x: x.max() - x.min())
top_inflation_countries = df_inflation_variation.nlargest(10).index
df_top_inflation_countries = df_economy_melted[df_economy_melted['Country'].isin(top_inflation_countries)].sort_values(by=['Year'])

fig_inflation_countries = px.line(df_top_inflation_countries, x='Year', y='Inflation', color='Country',
                                  title='🌍 Comparação dos Países com Maior Variação da Inflação ao longo dos Anos',
                                  labels={'Inflation': 'Inflação', 'Year': 'Ano'},
                                  markers=True)
fig_inflation_countries.update_traces(line=dict(width=2))
fig_inflation_countries.show()

# Padronizando os dados e fazendo correlação

In [61]:
print('Países no dataset de clima:', df_melted['Country'].unique())
print('Países no dataset de inflação:', df_economy['Country'].unique())

Países no dataset de clima: ['Afghanistan, Islamic Rep. of' 'Albania' 'Algeria' 'American Samoa'
 'Andorra, Principality of' 'Angola' 'Anguilla' 'Antigua and Barbuda'
 'Argentina' 'Aruba, Kingdom of the Netherlands' 'Australia' 'Austria'
 'Bahamas, The' 'Bahrain, Kingdom of' 'Bangladesh' 'Barbados' 'Belize'
 'Benin' 'Bhutan' 'Bolivia' 'Botswana' 'Brazil' 'British Virgin Islands'
 'Brunei Darussalam' 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cabo Verde'
 'Cambodia' 'Cameroon' 'Canada' 'Central African Rep.' 'Chad' 'Chile'
 'China, P.R.: Hong Kong' 'China, P.R.: Macao' 'China, P.R.: Mainland'
 'Colombia' 'Comoros, Union of the' 'Congo, Dem. Rep. of the'
 'Congo, Rep. of' 'Cook Islands' 'Cuba' 'Cyprus' 'Denmark' 'Djibouti'
 'Dominica' 'Dominican Rep.' 'Ecuador' 'Egypt, Arab Rep. of' 'El Salvador'
 'Equatorial Guinea, Rep. of' 'Eswatini, Kingdom of'
 'Falkland Islands (Malvinas)' 'Faroe Islands' 'Fiji, Rep. of' 'Finland'
 'France' 'French Polynesia' 'Gabon' 'Gambia, The' 'Germany' 'Ghana'
 'Gib

In [62]:
corrections = {
    'Afghanistan': 'Afghanistan, Islamic Rep. of',
    'China, People\'s Republic of': 'China, P.R.: Mainland',
    'Congo, Republic of ': 'Congo, Rep. of',
    'Congo, Dem. Rep. of the': 'Congo, Dem. Rep. of the',
    'Korea, Republic of': 'Korea, Rep. of',
    'Eswatini': 'Eswatini, Kingdom of',
    'United States' : 'United States',
    'Türkiye, Republic of': 'Turkey',
    'Hong Kong SAR' : 'China, P.R.: Hong Kong',
    'Macao SAR' : 'China, P.R.: Macao',
    'North Macedonia ' : 'North Macedonia, Republic of ',
    'São Tomé and Príncipe': 'São Tomé and Príncipe, Dem. Rep. of'
}

df_economy['Country'] = df_economy['Country'].replace(corrections)

# Verificando as correções
print('Países no dataset de clima:', df_melted['Country'].unique())
print('Países no dataset de inflação:', df_economy['Country'].unique())

Países no dataset de clima: ['Afghanistan, Islamic Rep. of' 'Albania' 'Algeria' 'American Samoa'
 'Andorra, Principality of' 'Angola' 'Anguilla' 'Antigua and Barbuda'
 'Argentina' 'Aruba, Kingdom of the Netherlands' 'Australia' 'Austria'
 'Bahamas, The' 'Bahrain, Kingdom of' 'Bangladesh' 'Barbados' 'Belize'
 'Benin' 'Bhutan' 'Bolivia' 'Botswana' 'Brazil' 'British Virgin Islands'
 'Brunei Darussalam' 'Bulgaria' 'Burkina Faso' 'Burundi' 'Cabo Verde'
 'Cambodia' 'Cameroon' 'Canada' 'Central African Rep.' 'Chad' 'Chile'
 'China, P.R.: Hong Kong' 'China, P.R.: Macao' 'China, P.R.: Mainland'
 'Colombia' 'Comoros, Union of the' 'Congo, Dem. Rep. of the'
 'Congo, Rep. of' 'Cook Islands' 'Cuba' 'Cyprus' 'Denmark' 'Djibouti'
 'Dominica' 'Dominican Rep.' 'Ecuador' 'Egypt, Arab Rep. of' 'El Salvador'
 'Equatorial Guinea, Rep. of' 'Eswatini, Kingdom of'
 'Falkland Islands (Malvinas)' 'Faroe Islands' 'Fiji, Rep. of' 'Finland'
 'France' 'French Polynesia' 'Gabon' 'Gambia, The' 'Germany' 'Ghana'
 'Gib

In [63]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [64]:
# Função para criar o gráfico combinado
def plot_inflation_temperature(df_melted, df_economy, country):
    """Cria um gráfico combinado de inflação e temperatura para um país específico."""

    # Usando df_melted para obter os dados climáticos, ele contém a coluna do ano
    df_climate_country = df_melted[df_melted['Country'] == country]
    df_economy_country = df_economy[df_economy['Country'] == country]

    # Derretendo o DataFrame df_economy_country para criar uma coluna 'Ano'
    years = [col for col in df_economy_country.columns if col.isdigit()]
    df_economy_country = df_economy_country.melt(id_vars=['Country', 'Indicator'], value_vars=years, var_name='Year', value_name='Inflation')
    df_economy_country['Year'] = df_economy_country['Year'].astype(int) # Convert 'Year' to int

    #o dataset sobre inflação começa apartir de 1980
    df_climate_country = df_climate_country[df_climate_country['Year'] >= 1980]
    df_economy_country = df_economy_country[df_economy_country['Year'] >= 1980]


    if df_climate_country.empty or df_economy_country.empty:
        print(f"Dados insuficientes para {country}. Pulando...")
        return

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(go.Scatter(x=df_economy_country['Year'], y=df_economy_country['Inflation'], name='Inflação (%)', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=df_climate_country['Year'], y=df_climate_country['Temperature_Change'], name='Mudança de Temperatura (°C)', line=dict(color='red')), secondary_y=True)

    fig.update_layout(title=f'Inflação e Temperatura ao Longo do Tempo - {country}', xaxis_title='Ano', yaxis_title='Inflação (%)', yaxis2_title='Mudança de Temperatura (°C)')
    fig.show()

In [65]:
paises_dependencia_agricultura = [
    'Australia', 'Bangladesh', 'Burkina Faso', 'Canada', 'Chad', 'Guinea',
    'Guinea-Bissau', 'Haiti', 'India', 'Indonesia', 'Liberia', 'Mali',
    'Myanmar', 'Nepal', 'Papua New Guinea', 'Senegal', 'Sierra Leone',
    'Thailand', 'Uganda', 'Zimbabwe'
]

In [66]:
# Gerando gráficos para países com dependência da agricultura
print("Gráficos para países com grande dependência da agricultura:")
for country in paises_dependencia_agricultura:
    plot_inflation_temperature(df_melted, df_economy, country)


Gráficos para países com grande dependência da agricultura:
